In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt

from empowered.utils.helpers import get_sql_client

In [ ]:
sql_client = get_sql_client()
pd.set_option('display.max_rows', None)   # Show all rows
pd.set_option('display.max_columns', None) # Show all columns
pd.set_option('display.width', 1000)      # Wider display
pd.set_option('display.max_colwidth', None)

In [ ]:
estimates = sql_client.execute("SELECT * FROM fullCensusEstimate;")
estimates_df = pd.DataFrame(estimates)
estimates_df

Current data format is not suitable for similarity analysis in algorithms like k-means clustering. We need to pivot into a form where columns are variables and rows are places such that each row, column pair is an estimate.

In [ ]:
estimates_df_pivot = estimates_df.pivot_table(
    index='place_fips',
    columns='variable_id',
    values='estimate'
).reset_index()
estimates_df_pivot.columns.name = None

estimates_df_pivot.head(1)

In [ ]:
estimates_df_pivot.isnull().sum()
estimates_df.fillna(0)

In [ ]:
correlation_matrix = estimates_df_pivot.corr()
correlation_matrix

We want to see what features are the most correlated.

In [ ]:
# Solution from https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas
def get_correlation_pairs(correlation_matrix):
  correlation_matrix_abs = correlation_matrix.abs()
  correlation_matrix_flat = correlation_matrix_abs.unstack()
  sorted_correlations = correlation_matrix_flat.sort_values(kind="quicksort")
  return sorted_correlations

In [ ]:
sorted_correlations_unnorm = get_correlation_pairs(correlation_matrix=correlation_matrix)[::-1]
sorted_correlations_unnorm

We find that many of the variables are correlated, as expected considering many of them are subsets of each other. For example, all variables are a subset of B1003_001E, the total population. To be sure we will see how the features interact visually.

Generating feature subsets to not overload plotting software.  

In [ ]:
def pairplot(data, subset):
  sns.pairplot(data[subset])
  plt.show()

In [ ]:
housing_features = [
    'B25001_001E',  # Total housing units
    'B25002_001E',  # Total occupied/vacant housing units
    'B25010_001E',  # Average household size
    'B25064_001E',  # Median gross rent
    'B25070_001E',  # Gross rent as % of household income
    'B25077_001E'   # Median home value
]

income_features = [
    'B19013_001E',  # Median household income
    'B19301_001E',  # Per capita income
    'B19001_001E',  # Total households (income distribution)
    'B19001_002E','B19001_003E','B19001_004E','B19001_005E','B19001_006E',
    'B19001_007E','B19001_008E','B19001_009E','B19001_010E','B19001_011E',
    'B19001_012E','B19001_013E','B19001_014E','B19001_015E','B19001_016E',
    'B19001_017E'
]

education_features = [
    'B15003_017E',  # High school graduate
    'B15003_022E',  # Bachelor’s degree
    'B15003_023E',  # Master’s degree
    'B15003_025E'   # Doctorate degree
]

school_enrollment_features = [
    'B14007_001E','B14007_002E','B14007_003E','B14007_004E','B14007_005E',
    'B14007_006E','B14007_007E','B14007_008E','B14007_009E','B14007_010E',
    'B14007_011E','B14007_012E','B14007_013E','B14007_014E','B14007_015E',
    'B14007_016E','B14007_017E','B14007_018E','B14007_019E'
]

poverty_school_features = [
    'B14006_001E','B14006_002E','B14006_003E','B14006_004E','B14006_005E',
    'B14006_006E','B14006_007E','B14006_008E','B14006_009E','B14006_010E',
    'B14006_011E','B14006_012E','B14006_013E','B14006_014E','B14006_015E',
    'B14006_016E','B14006_017E','B14006_018E','B14006_019E','B14006_020E',
    'B14006_021E'
]

children_assistance_features = [
    'B09010_001E','B09010_002E','B09010_003E','B09010_004E','B09010_005E',
    'B09010_006E','B09010_007E','B09010_008E','B09010_009E','B09010_010E',
    'B09010_011E','B09010_012E','B09010_013E'
]

poverty_sex_age_features = [
    'B17001_001E','B17001_002E','B17001_003E','B17001_004E','B17001_005E',
    'B17001_006E','B17001_007E','B17001_008E','B17001_009E','B17001_010E',
    'B17001_011E','B17001_012E','B17001_013E','B17001_014E','B17001_015E',
    'B17001_016E','B17001_017E','B17001_018E','B17001_019E','B17001_020E',
    'B17001_021E','B17001_022E','B17001_023E','B17001_024E','B17001_025E',
    'B17001_026E','B17001_027E','B17001_028E'
]

In [ ]:
pairplot(estimates_df_pivot, housing_features)

In [ ]:
pairplot(estimates_df_pivot, education_features)

In [ ]:
count_per_column = (estimates_df_pivot == -666666666).sum()
print(count_per_column)


Census api treats -666666666 as a placeholder, therefore we are converting it to 0 so it does not skew any points.

In [ ]:
estimates_df_pivot.replace(-666666666, 0, inplace=True)

Current values are all over the place and will ruin the clustering. Therefore we will normalize population and counts by total population size and then standarize all results.

In [ ]:
estimates_df_norm = estimates_df_pivot.copy()

pop_col = 'B01003_001E'

# List of count variables to normalize by population
count_vars = [
    # Children & Public Assistance
    'B09010_001E', 'B09010_002E', 'B09010_003E', 'B09010_004E',
    'B09010_005E', 'B09010_006E', 'B09010_007E', 'B09010_008E',
    'B09010_009E', 'B09010_010E', 'B09010_011E', 'B09010_012E',
    'B09010_013E',
    
    # School Enrollment
    'B14007_001E', 'B14007_002E', 'B14007_003E', 'B14007_004E',
    'B14007_005E', 'B14007_006E', 'B14007_007E', 'B14007_008E',
    'B14007_009E', 'B14007_010E', 'B14007_011E', 'B14007_012E',
    'B14007_013E', 'B14007_014E', 'B14007_015E', 'B14007_016E',
    'B14007_017E', 'B14007_018E', 'B14007_019E',
    
    # Poverty Status by School Enrollment
    'B14006_001E', 'B14006_002E', 'B14006_003E', 'B14006_004E',
    'B14006_005E', 'B14006_006E', 'B14006_007E', 'B14006_008E',
    'B14006_009E', 'B14006_010E', 'B14006_011E', 'B14006_012E',
    'B14006_013E', 'B14006_014E', 'B14006_015E', 'B14006_016E',
    'B14006_017E', 'B14006_018E', 'B14006_019E', 'B14006_020E',
    'B14006_021E',
    
    # Poverty Status by Sex and Age
    'B17001_001E', 'B17001_002E', 'B17001_003E', 'B17001_004E',
    'B17001_005E', 'B17001_006E', 'B17001_007E', 'B17001_008E',
    'B17001_009E', 'B17001_010E', 'B17001_011E', 'B17001_012E',
    'B17001_013E', 'B17001_014E', 'B17001_015E', 'B17001_016E',
    'B17001_017E', 'B17001_018E', 'B17001_019E', 'B17001_020E',
    'B17001_021E', 'B17001_022E', 'B17001_023E', 'B17001_024E',
    'B17001_025E', 'B17001_026E', 'B17001_027E', 'B17001_028E'
]

In [ ]:
# apply normalization
for col in count_vars:
    if col in estimates_df_norm.columns:
        estimates_df_norm[col] = estimates_df_norm[col] / estimates_df_norm[pop_col]

In [ ]:
estimates_df_norm.head()

In [ ]:
correlation_matrix_norm = estimates_df_norm.corr()

In [ ]:
sorted_correlations_norm = get_correlation_pairs(correlation_matrix=correlation_matrix_norm)[::-1]
sorted_correlations_norm

In [ ]:
scaler = StandardScaler()
scale_cols = estimates_df_norm.columns[1::]

In [ ]:
estimates_df_normscaled = pd.DataFrame(scaler.fit_transform(estimates_df_norm[scale_cols]), columns=scale_cols, index=estimates_df_norm.index,)
estimates_df_normscaled["place_fips"] = estimates_df_norm["place_fips"]

In [ ]:
estimates_df_normscaled.head(1)